# What is Featuristic?

![featuristic_logo](_static/logo.png "Featuristic")

**Featuristic** uses Genetic Algorithms to automate the process of **feature engineering** and **feature selection**, enhancing the performance of machine learning models by optimizing their predictive capabilities.

## Understanding Genetic Feature Synthesis

Featuristic uses symbolic regression to intelligently derive interpretable mathematical formulas, which are then used to create new features from your dataset.

Initially, Featuristic creates a diverse population of formulas using fundamental mathematical operators such as `add`, `subtract`, `sin`, `tan`, `square`, `sqrt`, and more.

For instance, a formula generated by Featuristic might look like this: `(square(feature_1) - abs(feature_2)) * feature_3`.

Next, Featuristic assesses the efficacy of these formulas by quanifying how effectively the features correlate with the target variable. Those formulas yielding features with the strongest correlations are then selected and recombined using a genetic algorithm to produce offspring, as illustrated below.

![Symbolic Regression Example](_static/symbolic_regression_example.png "Symbolic Regression Example")

These offspring may also undergo point mutations, which causes alterations to random operators within the formula. This process introduces slight variations to the formulas, enhancing the diversity of the population and potentially leading to the discovery of novel and more effective feature representations.

![Mutation Example](_static/mutation_example.png "Mutation Example")

This iterative process continues across multiple generations, continually refining the population of formulas with the goal of generating features that exhibit strong correlations with the target variable.

## Quickstart

Below is a simple example of how Featuristic performs automated feature engineering and selection on the widely used `cars` dataset.

Featuristic operates in two distinct steps:

1. **Genetic Feature Synthesis:** In this initial phase, Featuristic intelligently evolves new features through a form of symbolic regression. This involves the generation of mathematical expressions using genetic algorithms. These expressions are designed to capture complex relationships within the dataset.

2. **Genetic Feature Selection:** Following the creation of new features, Featuristic employs a Genetic Feature Selection algorithm. This algorithm searches through the newly formed feature space to identify the most optimal subset of features. The objective is to maximize predictive accuracy while minimizing the number of features required for model training.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
import featuristic as ft
import numpy as np

np.random.seed(8888)

print(ft.__version__)

0.1.1


### Load the Data

In [2]:
X, y = ft.fetch_cars_dataset()

X.head()

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin
0,307.0,8,130.0,3504,12.0,70,1
1,350.0,8,165.0,3693,11.5,70,1
2,318.0,8,150.0,3436,11.0,70,1
3,304.0,8,150.0,3433,12.0,70,1
4,302.0,8,140.0,3449,10.5,70,1


In [3]:
y.head()

0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64

### Genetic Feature Synthesis

Now, let's dive into the fun part: using Genetic Feature Synthesis to automatically engineer new features from our dataset!

Before we proceed, it's important to ensure a robust evaluation of our model's performance. To achieve this, we'll first split our dataset into training and testing sets. The training set will be used to train our model, while the testing set will remain unseen during the training process and will serve as an independent dataset to evaluate the model's performance.

Once our data is appropriately split, we'll initiate the Genetic Feature Synthesis process. We've configured the genetic algorithm to  synthesize 5 new features for us. This entails evolving a population consisting of 200 individuals iteratively over 100 generations. To ensure optimal performance, we've set the genetic algorithm to halt early if it fails to improve upon the best feature identified within 25 generations. Additionally, for enhanced computational efficiency, we've designated `n_jobs` as -1, enabling concurrent execution across all available CPUs on our computer.

With everything set up, we simply call the `fit` function to generate our new features.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

synth = ft.GeneticFeatureSynthesis(
    num_features=5,
    population_size=200,
    max_generations=100,
    early_termination_iters=25,
    parsimony_coefficient=0.0025,
    n_jobs=-1,
)

synth.fit(X_train, y_train)

None

Creating new features...:  27%|████▎           | 27/100 [00:11<00:29,  2.45it/s]


Next, we call the `transform` function to generate a dataframe containing our new features. By default, the `GeneticFeatureSynthesis` class will return both the original features and the newly synthesised features. However, we return just the new features by setting the `return_all_features` argument to False when we create the class. 

We can also combine both the `fit` and `transform` steps into one step by calling `fit_transform` instead.

In [5]:
generated_features = synth.transform(X_train)

generated_features.head()

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin,feature_0,feature_7,feature_1,feature_8,feature_2
0,122.0,4,86.0,2220,14.0,71,1,0.072622,1.049748e+09,206072.467213,1.966114e+10,-210512.467213
1,200.0,6,88.0,3060,17.1,81,1,0.056756,1.411911e+09,212173.605000,3.596968e+10,-218293.605000
2,302.0,8,129.0,3725,13.4,79,1,0.035533,8.047110e+08,125248.778146,1.085258e+10,-132698.778146
3,302.0,8,140.0,4294,16.0,72,1,0.020243,4.611275e+08,84692.278146,2.032975e+08,-93280.278146
4,120.0,4,97.0,2506,14.5,72,3,0.059434,1.160646e+09,221442.800000,1.673793e+10,-226454.800000


Our newly engineered features currently have generic names. However, since Featuristic synthesizes these features by the applying mathematical expressions to the data, we can look at the underlying formulas responsible for each feature's creation.

In [6]:
info = synth.get_feature_info()
info["formula"].iloc[0]

'(square(abs((square(-(model_year)) / abs(weight)))) / model_year)'

### Feature Selection

Following the synthesis of new features, the next step involves using another genetic algorithm for feature selection. This process sifts through the pool of features to identify the subset that optimally contributes to predictive performance while minimizing redundancy. 

### Define the Cost Function

We set up a custom objective function that the Genetic Feature Selection algorithm will use to quantify how well the subset of features predicts the target. Please note that the function should return a value to **minimize** so a **smaller value is better**. If you want to maximize a metric, you should multiply the output of your objective_function by -1, as shown in the example below.

In [7]:
def objective_function(X, y):
    model = LinearRegression()
    scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")
    return scores.mean() * -1

Next, we set up the Genetic Feature Selector. We've configured the genetic algorithm to evolve a population consisting of 200 individuals iteratively over 100 generations. To ensure optimal performance, we've set the genetic algorithm to halt early if it fails to improve upon the best feature set identified within 25 generations. Additionally, for enhanced computational efficiency, we've set `n_jobs` as -1, enabling concurrent execution across all available CPUs on our computer.

In [8]:
selector = ft.GeneticFeatureSelector(
    objective_function,
    population_size=200,
    max_generations=50,
    early_termination_iters=25,
    n_jobs=-1,
)

selector.fit(generated_features, y_train)

selected_features = selector.transform(generated_features)

print(len(selected_features.columns))

Optimising feature selection...:  56%|█████▌    | 28/50 [00:05<00:04,  5.44it/s]

6


### The New Features

Let's print out the selcted features to see what the Genetic Feature Selection algorithm kept. You can see below that featuristic has kept two of the original features ("cylinders" and "origin") and has kept four of the features created via the Genetic Feature Synthesis.

In [9]:
selected_features.head()

,cylinders,origin,feature_0,feature_7,feature_1,feature_2
0,4,1,0.072622,1.049748e+09,206072.467213,-210512.467213
1,6,1,0.056756,1.411911e+09,212173.605000,-218293.605000
2,8,1,0.035533,8.047110e+08,125248.778146,-132698.778146
3,8,1,0.020243,4.611275e+08,84692.278146,-93280.278146
4,4,3,0.059434,1.160646e+09,221442.800000,-226454.800000


### Training a Model

Now that we've selected our features, let's see whether they actually help our model's predictive performance on our test data set. We'll start off with the original features as a baseline.

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)
original_mae = mean_absolute_error(y_test, preds)
original_mae

2.581708285266024

And now, let's see how the model performs with our synthesised feature set.

In [11]:
model = LinearRegression()
model.fit(selected_features, y_train)
test_features = selector.transform(synth.transform(X_test))
preds = model.predict(test_features)
featuristic_mae = mean_absolute_error(y_test, preds)

In [12]:
print(f"Original MAE: {original_mae}")
print(f"Featuristic MAE: {featuristic_mae}")
print(f"Improvement: {round((1 - (featuristic_mae / original_mae))* 100, 1)}%")

Original MAE: 2.581708285266024
Featuristic MAE: 2.243230501011035
Improvement: 13.1%


The new features generated / selected by the Genetic Feature Synthesis have reduced our mean absolute error &#128512;

<br />
<br />